### MechInterp

In [ ]:
query, key, value, attn, output

In [ ]:
[2, 5] x [5, 3] = [2, 3]

[2, 3] x [3, 5]

In [ ]:
model.W_E

In [ ]:
blank = linear_probe[..., 0]

In [ ]:
not_empty = (linear_probe[..., 1] + linear_probe[..., 2])/2

In [ ]:
empty_vs_not_empty = blank - not_empty

In [1]:
from einops import einsum

In [ ]:
def calculate_similarity(weight, feature):
    weight /= weight.norm()
    feature /= feature.norm()
    
    return einsum(
        weight, feature,
        ""
    )

In [ ]:
_, cache = model.run_with_cache(past_moves)

In [2]:
from transformer_lens.utils import get_act_name

In [3]:
hook_name = get_act_name("out", 4)

In [ ]:
top_neurons = cache[hook_name].std(dim=[0, 1]).argsort(descending=True)[:10]

In [4]:
heatmap_blanks = []

In [ ]:
for neuron_idx in top_neurons:
    heatmap_blanks.append(calculate_similiarty(
        model.W_out[4, neuron_idx, :],
        blank_dir
    ))

In [ ]:
step 1: diverse set of data
step 2: record the activation of the target neuron
step 3: plot
step 4: 

In [ ]:
W_pos = model.W_pos

In [ ]:
one = W_pos[:, 0]
one /= one.norm()

two = W_pos[:, 1]
two /= two

In [ ]:
similarity = einsum(
    one, two,
    ""
)

In [ ]:
mine = linear_probe[..., 2]
theirs = linear_probe[..., 1]

In [ ]:
mine_vs_theirs = mine - theirs

In [ ]:
_, cache = model.run_with_cache(
    board_history
)

In [6]:
attn_contributions = []

In [ ]:
for layer_idx in range(6):
    hook_name = get_act_name("attn_out", layer_idx)
    attn_out = cache[hook_name]
    attn_contributions.append(einsum(
        attn_out,
        mine_vs_theirs
    ))

In [7]:
import torch
from torch import nn
import torch.nn.functional as F

In [8]:
class Checkpoint(torch.autograd.Function):
    @staticmethod
    def forward(ctx, phony, recomputed, function, input):
        ctx.recomputed = recomputed
        with torch.no_grad():
            output = function(input)
        
        return output
    
    @staticmethod
    def backward(ctx, grad):
        output, input_leaf = ctx.recomputed.pop()
        
        with torch.enable_grad():
            torch.autograd.backward(output, input_leaf)
        
        return (None, None, None, input_leaf.grad)

In [ ]:
step 1: initialize global dist group
step 2: 

In [ ]:
step 1: cpu memory
step 2: gpu memory
step 3: move to gpu memory
step 4: launch kernel and do
step 5: move the result back to cpu
step 6: free gpu memory

In [ ]:
main thread > worker thread > task > cuda stream

In [ ]:
global dist, tensor, data, pipeline 

In [ ]:
def get_handlers(states):
    handlers, regulars = {}, {}
    
    for 

In [ ]:
class ModelStateHandler:
    def __init__(self, model):
        pass

In [ ]:
global dist
parallel group
set gpu
random seed

In [ ]:
register > cache > main memory > hard disk

In [ ]:
adding relu
sparity
importance
correlated vs anti-correlated features

In [ ]:
_, cache = model.run_with_cache(board_history)

In [ ]:
mine = linear_probe[..., 2]
theirs = linear_probe[..., 1]
mine_vs_theirs = mine - theirs

In [9]:
n_layers = 6

In [10]:
attn_contributions = []

In [ ]:
for layer_idx in range(n_layers):
    hook_name = get_act_name("attn_out", layer_idx)
    attn_out = cache[hook_name][2, 20]
    contribution = einsum(
        attn_out,
        mine_vs_theirs,
        ""
    )
    attn_contributions.append(contribution)

In [ ]:
step 1: prob[0] = sigmoid(logit0 - logit1)
step 2: logit0 = resid2 @ W_U[0], resid2 @ W[1]
step 3: logit0 - logit1 = resid2 @ (W_U[0] - W_U[1])
step 4: resid2 = resid1 @ ln2 @ W_OV^{2, 0}
step 5: resid1 @ ln2 @ W_OV^{2, 0} @ (W_U[0] - W_U[1])

ln2 @ W_OV^{2, 0} @ (W_U[0] - W_U[1])

In [ ]:
_, cache = model.run_with_cache(past_moves)

In [11]:
layer_idx = 3

In [12]:
hook_name = get_act_name("out", layer_idx)

In [ ]:
top_neurons = cache[hook_name].std

In [ ]:
step 1: 

In [ ]:
_, cache = model.run

In [ ]:
conditioned response, stimulus-evoked, population decoding, motor

In [14]:
def compute_total_memory(model):
    total_memory = 0    
    for p in model.parameters():
        total_memory += p.element_size() * p.numel()
    return total_memory

In [15]:
def get_handler(v):
    for handler_type, handler_cls in handler_registry:
        if isinstance(v, handler_type):
            return handler_cls(v)
    return None

In [ ]:
def get_handlers(states):
    handlers = {}
    remainders = {}
    
    for name, value in states:
        

In [ ]:
register > cache > dram > hard drisk

In [ ]:
int x{y}

In [16]:
world_size = 16

In [17]:
data_parallel_groups = []

In [18]:
num_pipeline_model_parallel_groups = 4

In [ ]:
for i in range(num_pipeline_model_parallel_groups):
    start_rank = i*

In [ ]:
rank, world_size, tensor_mod

In [ ]:
n_observations = env.action_space.

In [19]:
class DeepQNetwork(nn.Module):
    def __init__(self, n_observations, n_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_observations, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.Linear(128, n_actions)
        )
    
    def forward(self, x):
        return self.net(x)

In [20]:
from torch.distributions import Categorical

In [ ]:
class Agent(nn.Module):
    def __init__(self, n_observations, n_actions, n_hidden):
        super().__init__()
        self.actor_net = nn.Sequential(
            nn.Linear(n_observations, n_hidden)
            nn.Tanh(),
            nn.Linear(n_hidden, n_hidden),
            nn.Tanh(),
            nn.Linear(n_hidden, n_actions)
        )
        
        self.critic_net = nn.Sequential(
            nn.Linear(n_observations, n_hidden),
            nn.Tanh(),
            nn.Linear(n_hidden, n_hidden),
            nn.Tanh(),
            nn.Linear(n_hidden, 1),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        logits = self.actor_net(x)
        dist = Categorical(logits=logits)
        action = dist.sample()
        log_prob = dist.log_prob(action)
        entropy = dist.entropy()
        value = self.critic_net(x)
        return

In [ ]:
for i in range(tensor_model_parallel_size):
    start_rank = i*num_tensor_model_parallel_size
    end_rank = (i+1)*num_tensor_model_parallel_size
    
    for j in range(tensor_model_parallel_size):
        ranks = list(range(start_rank+j, end_rank, tensor_model_parallel+size ))

In [ ]:
w_in = model.W_in[5, : 1393]
w_out = model.W_out[5, 1393, :]

In [ ]:
def compute_consine_similarity(weight, feature):
    weight /= weight.norm()
    feature /= feature.norm()
    
    return einsum(
        weight, feature,
        ""
    )

In [ ]:
_, cache = model.run_with_cache(past_moves)

In [21]:
hook_name = get_act_name("out", 3)

In [22]:
hook_name

'blocks.3.hook_out'

In [ ]:
top_neurons = cache[hook_name].std(dim=[0, 1]).argsort(descending=True)[:10]

In [23]:
heatmap_blanks = []

In [ ]:
for neuron_idx in top_neurons:
    w_out = model.W_out[3, neuron_idx, :]
    w_out /= w_out.norm()
    
    blank_dir /= blank_dir.norm()
    
    heatmap_blanks.append(einsum(
        w_out,
        blank_dir
    ))

In [ ]:
target_tokens = tokens[1:]

In [ ]:
embed = cache["embed"]

In [ ]:
embed_attributions = einsum(
    embed[:, -1],
    model.W_U[:, target_tokens]
)

In [ ]:
memory coefficients = ReLU(resid[0] @ k_i)


values = memory_coefficient_i @ v_i

recalled_memory = sum(values) + bias